<a href="https://colab.research.google.com/github/beatLaboratory/TIMBRE/blob/main/extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing and downsampling data

Here we show how the behavioral, spiking, and LFP data can be imported from figshare and saved at 25 Hz sampling rate, thus reducing the memory requirements. For most people this notebook is not important, as we will only be working with the downsampled data (i.e., the files created by this notebook).

In [ ]:
!git clone https://github.com/beatLaboratory/TIMBRE.git

Cloning into 'TIMBRE'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 86 (delta 46), reused 36 (delta 14), pack-reused 0
Receiving objects: 100% (86/86), 3.07 MiB | 10.74 MiB/s, done.
Resolving deltas: 100% (46/46), done.


#Extract data

Now we will access each of 4 sessions, extracting the behavior, spiking, and LFP files, decimating them, and saving them as data*.mat files.

In [ ]:
import requests
import os
import io
import tempfile
import scipy.io as iomat
import get_data

repository_id = "24757638"  # Behavior_and_spiking_data_for_rats_running_a_3-arm_maze"
url = f"https://api.figshare.com/v2/articles/{repository_id}"

# Make the API request
response = requests.get(url)
files = response.json()['files']
n_channels = [385, 385, 256, 256]

# Loop through a set range
for i in range(4):  # Adjust the range as needed
    behav_file_pattern = f"rec0{i + 1}_BehavElectrDataLFP.mat"
    lfp_file_pattern = f"rec0{i + 1}.lfp"

    # Find the matching files
    behav_file = next((file for file in files if file['name'] == behav_file_pattern), None)
    lfp_file = next((file for file in files if file['name'] == lfp_file_pattern), None)
    
    # Initialize combined data dictionary
    full_data = {}
    
    if behav_file and lfp_file:
        print(f"Downloading files: {behav_file['name']} and {lfp_file['name']}")

        behav_response = requests.get(behav_file['download_url'])
        behav_filename = f"rec0{i + 1}_BehavElectrDataLFP.mat"
        with open(behav_filename, 'wb') as f:
            f.write(behav_response.content)
        print(f"Downloaded {behav_filename}")

        full_data['behav'] = iomat.loadmat(behav_filename)

       # Download LFP file
        lfp_response = requests.get(lfp_file['download_url'])
        lfp_filename = f"rec0{i + 1}.lfp"
        with open(lfp_filename, 'wb') as f:
            f.write(lfp_response.content)
        print(f"Downloaded {lfp_filename}")

        full_data['lfps'] = get_data.get_LFP(lfp_filename, n_channels[i], 1250)

        iomat.savemat(f"data0{i + 1}.mat", full_data)
